## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-02-01-13-00 +0000,wsj,Singapore’s Economy Records Stronger-Than-Expe...,https://www.wsj.com/economy/singapores-economy...
1,2026-01-02-01-04-07 +0000,bbc,Why everything from your phone to your PC may ...,https://www.bbc.com/news/articles/c1dzdndzlxqo...
2,2026-01-02-00-53-13 +0000,bbc,How the defence sector is battling a skills cr...,https://www.bbc.com/news/articles/clyd1lpp1lyo...
3,2026-01-02-00-48-00 +0000,wsj,The president veered between business and plea...,https://www.wsj.com/politics/policy/golf-peace...
4,2026-01-02-00-31-50 +0000,nypost,"Terrifying video, cockpit audio captures momen...",https://nypost.com/2026/01/01/us-news/terrifyi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2493/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,new,43
26,year,37
110,trump,25
90,fire,17
88,swiss,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
109,2026-01-01-13-52-12 +0000,nypost,Horrific video shows the moment deadly New Yea...,https://nypost.com/2026/01/01/world-news/new-y...,154
124,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...,151
157,2026-01-01-07-42-14 +0000,nypost,"40 people dead, 100 hurt after fire rips throu...",https://nypost.com/2026/01/01/world-news/swiss...,150
17,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,148
87,2026-01-01-17-09-57 +0000,nypost,Foreign tourists among 47 confirmed dead in Ne...,https://nypost.com/2026/01/01/world-news/many-...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
109,154,2026-01-01-13-52-12 +0000,nypost,Horrific video shows the moment deadly New Yea...,https://nypost.com/2026/01/01/world-news/new-y...
179,57,2026-01-01-02-19-53 +0000,nypost,"Trump calls for ‘peace on Earth,’ vows to ‘get...",https://nypost.com/2025/12/31/us-news/trump-ca...
56,52,2026-01-01-20-02-35 +0000,nypost,‘Tax the rich’ chant breaks out as Bernie Sand...,https://nypost.com/2026/01/01/us-news/tax-the-...
57,33,2026-01-01-19-55-28 +0000,nypost,Son of late New York musician was ready to com...,https://nypost.com/2026/01/01/us-news/grieving...
117,31,2026-01-01-12-48-00 +0000,wsj,President Trump delayed by a year tariff incre...,https://www.wsj.com/economy/trade/trump-rolls-...
95,29,2026-01-01-15-40-44 +0000,nyt,Trump Must Return Command of California Nation...,https://www.nytimes.com/2025/12/31/us/trump-na...
124,28,2026-01-01-11-13-29 +0000,cbc,"Around 40 dead, more than 100 injured in blaze...",https://www.cbc.ca/news/world/bar-fire-switzer...
173,27,2026-01-01-03-28-50 +0000,cbc,U.S. military says 8 killed in strikes on 5 mo...,https://www.cbc.ca/news/world/us-venezuela-dru...
177,26,2026-01-01-02-29-38 +0000,nyt,CI.A. Says Ukraine Did Not Target Putin’s Home...,https://www.nytimes.com/2025/12/31/us/politics...
181,25,2026-01-01-02-10-54 +0000,latimes,Puppy found in trash compactor in Gardena. Pol...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
